# Training Resnet50 for Head's Position Detection
After preparing the `margined_cropped_images`, and `data.csv`, now we are able to feed them to a model to predict Hexbugs' heads' position.
The approach that we managed to work with was using ***transfer learning*** to build a better model. For this, we used pre-trained `Resnet50` model, which is trained on `CoCo` dataset. This helps us to extract feature better. Hoever, top layers of the `resnet` model are used for classification tasks. To modify the model to suit our problem, we removed tha last `Dense` layer and added two other `Dense` layers to build a regressor with only ***two*** outputs for `(x, y)`.
After modifying the model, we can train it over our dataset, and evaluate its performance on the validation set.
At last, we save the model for further use cases.

### Essential Imports

* `cv2` is required to read and write images.
* `json` is used to open `json` files.
* `random` is used to pick random samples for model evaluation.
* `logging` is used to ignore `warning` messages. (not mandatory)
* `numoy` is cruicial to work with images.
* `pandas` is used to read the `data.csv` file.
* `tensorflow` is used to create model. it's used as background for `Keras`.
* `sklearn` is used to split dataset to `train` and `test` sets.

In [27]:
import os
import cv2
import json
import random
import logging
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

tf.get_logger().setLevel(logging.ERROR)

### Reading data from `data.csv`

In [2]:
data = pd.read_csv('data.csv').drop(['Unnamed: 0'], axis=1).sort_values('ID', ascending=True)
data.head()

,CroppedHexBugCoordinationX,CroppedHexBugCoordinationY,OriginalBoxCoordinationX1,OriginalBoxCoordinationX2,OriginalBoxCoordinationY1,OriginalBoxCoordinationY2,Path,ID
497,197,95,923,1223,91,391,cropped_bugs/training04/frame0.jpg,0
396,41,31,591,891,218,518,cropped_bugs/training05/frame0.jpg,0
201,90,154,140,440,374,674,cropped_bugs/training01/frame0.jpg,0
100,197,95,908,1208,581,881,cropped_bugs/training03/frame0.jpg,0
302,104,175,66,366,263,563,cropped_bugs/training06/frame0.jpg,0


In [3]:
# Load the images
images = []
y = []

# Iterates over the `data.csv` file's rows to read images and annotatopns.
for idx, row in data.iterrows():
    img_path = f'./{row.Path}'
    annotations = [int(row.CroppedHexBugCoordinationX), int(row.CroppedHexBugCoordinationY)]
    # Reads the image
    img = load_img(img_path)
    # Converts the image to a numpy array
    np_img = np.array(img)
    # Adds the image to a list of images.
    images.append(np_img)
    # Adds the annotations to y.
    y.append(annotations)
    
X = np.array(images)
y = np.array(y)

print(X.shape)
print(y.shape)

(498, 300, 300, 3)
(498, 2)


In [4]:
# Splits the dataset to `train` and `test` sets with the given ratio.
# Also, shuffles the data to prevent feeding the model same data over different runs.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [5]:
print(f'X train shape: {X_train.shape}')
print(f'X test shape: {X_test.shape}')
print(f'y train shape: {y_train.shape}')
print(f'y test shape: {y_test.shape}')

X train shape: (398, 300, 300, 3)
X test shape: (100, 300, 300, 3)
y train shape: (398, 2)
y test shape: (100, 2)


In [6]:
# Creates a pre-trained model instance from class `ResNet50`.
# This model doesn't include the last classification dense layer.
# The given `input_shape` should be the same as our dataset images.
resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=X_train[0].shape)

# Iterates over each layer in the model and makes them to be trainable.
for layer in resnet50.layers:
    layer.trainable = True

# Adds two dense layers for regression problem.
# It basically uses the output of the `resnet50` model to feed the dense layers.
x = resnet50.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# The last layer should be the layer that computes the (x, y) coordinations.
output_layer = Dense(2, activation='linear')(x)

# Create the model based on the input and output layer.
model = Model(inputs=resnet50.input, outputs=output_layer)

2023-04-26 01:01:00.507196: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-26 01:01:00.526702: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-26 01:01:00.526829: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

### Visualization
For better visualization, we use [tensorboard](https://www.tensorflow.org/tensorboard/get_started). It helps us to plot the figures real-time and analyze the model performance.

In [6]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# We need datetime to name our checkpoints.
import datetime

# Clear any logs from previous runs
!rm -rf ./logs/

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Creates a callback, which then will be called by the model during training process.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

/usr/bin/zsh: /home/farzam/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)


### Saving the best model
During the training process, we need to track the model's performance and make sure that at the end of the training process, we have the best model.
To do so, we used a custom `callback` that checks if the `val_loss` has been decreased over the past epoch. In that case, it will simply save the model to the provided directory, which can be used to load and evaluate later.
This call back is then called by the model during the training process.

In [7]:
# Specifies the checkpoint directory
checkpoint_filepath = './checkpoint_callback'

# Creates an instance from the ModelCheckpoint class.
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [8]:
# Since we are dealing with regression problem, it's recommended to use `mse`.
# Also we use `Adam` as our optimizer.
model.compile(loss='mse', optimizer='adam')

NameError: name 'model' is not defined

In [ ]:
# Prints a summary of the compiled model
model.summary()

### Training the model
Now that we have everything ready, we can train the model. we pass the `validation_data` containing the `X_test` and `y_test` to the model as well.
Also, for above mentioned checkopints, we need to call those callbacks.

In [32]:
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test), callbacks=[tensorboard_callback, model_checkpoint_callback])

In [14]:
# Plotting the tensorboard data in jupyter notebook
%tensorboard --logdir logs/fit

In [9]:
def compute_error(y, y_pred):
    """
    Since the competion computes the error based on Euclidian distances, this function computes
    the error for predicted and ground truth results.
    
    @param: list y. a (1, 2) list with original coordinations.
    @param: list y_pred. a (1, 2) list containing the predicted coordinations.
    
    @return: float error. The Euclidian distance between these two given datapoints.
    """
    error = np.sqrt(np.abs(y[0] - y_pred[0]) ** 2 + np.abs(y[1] - y_pred[1]) ** 2)
    return error

In [10]:
def compute_original_coordination(predicted_coordination, original_box_coordination):
    """
    Computes the coordinations of the predicted Hexbug's head's coordination in the original image
    using the bounding box coordinations.
    
    @param: list predicted_coordination. a (1, 2) list with (x, y) format.
    @param: list original_box_coordination. a (1, 2) list with (x, y) format of bounding box.
    
    @return: list. Coordinations of the predicted Hexbug's head's coordination in the original image
    """
    return (predicted_coordination[0] + original_box_coordination[0], predicted_coordination[1] + original_box_coordination[1])

### Loading the best model

In [14]:
# reconstructed_model = tf.keras.models.load_model("checkpoint_callback")
reconstructed_model = tf.keras.models.load_model("resnet50_trained_model_data/v1.0/checkpoint_callback")

### Sample Evaluation
Now that we have successfully trained the model, let's see its performance on unseen data.

In [15]:
num_samples = 10

# Chooses random indices
random_samples_indices = random.sample(range(0, len(X_test)), num_samples)

# Iterates over each index and retrieves the image and annotaion
for sample_index in random_samples_indices:
    sample_img = X_test[sample_index]
    
    # Configs for drawing a circle on the image.
    center = y_test[sample_index]
    radius = 5
    # Color map is BGR
    color = (0, 255, 0)
    color_pred = (0, 0, 255)
    thickness = 5
    
    # Since we predict only one image at a time, we need to expand its dimention
    # to fit the input layer of our model.
    expanded_img = np.expand_dims(sample_img, axis=0)
    
    # Predicts the coordinations
    predicted = list(map(int, reconstructed_model.predict(expanded_img)[0]))
    
    # Computes the error
    error = round(compute_error(center, predicted), 3)
    
    print(f'y_predicted: {predicted}, y: {center}, Error: {error}')
    
    # Draws a circle centered in the correct coordination.
    img_lv1 = cv2.circle(sample_img, center, radius, color, thickness)
    # Draws a circle centered in the predicted coordination.
    img_lv2 = cv2.circle(img_lv1, predicted, radius, color_pred, thickness)
    
    # Shows the output image.
    cv2.imshow(f'Sample No. {sample_index}, error: {error}', img_lv2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

2023-04-28 20:06:23.367925: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


1/1 [==============================] - 2s 2s/step


2023-04-28 20:06:24.104231: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


y_predicted: [23, 161], y: [ 23 163], Error: 2.0
1/1 [==============================] - 0s 12ms/step
y_predicted: [44, 158], y: [ 44 157], Error: 1.0
1/1 [==============================] - 0s 12ms/step
y_predicted: [23, 33], y: [26 30], Error: 4.243
1/1 [==============================] - 0s 13ms/step
y_predicted: [36, 186], y: [ 37 185], Error: 1.414
1/1 [==============================] - 0s 12ms/step
y_predicted: [72, 51], y: [75 29], Error: 22.204
1/1 [==============================] - 0s 12ms/step
y_predicted: [235, 196], y: [236 196], Error: 1.0
1/1 [==============================] - 0s 12ms/step
y_predicted: [29, 54], y: [29 54], Error: 0.0
1/1 [==============================] - 0s 12ms/step
y_predicted: [46, 26], y: [47 27], Error: 1.414
1/1 [==============================] - 0s 12ms/step
y_predicted: [49, 27], y: [49 28], Error: 1.0
1/1 [==============================] - 0s 11ms/step
y_predicted: [37, 268], y: [ 38 267], Error: 1.414


### Sample for coordination convertion

In [29]:
num_samples = 10

random_samples_indices = random.sample(range(0, len(data)), num_samples)

for sample_id in random_samples_indices:
    sample_img_path = data.iloc[sample_id].Path

    # Constructs the path to the original frame
    original_image_path = f"Videos/{sample_img_path.split('/')[1]}/{sample_img_path.split('/')[2]}"

    sample_img = np.array(load_img(sample_img_path))

    expanded_img = np.expand_dims(sample_img, axis=0)

    predicted = list(map(int, reconstructed_model.predict(expanded_img)[0]))

    radius = 10
    color = (0, 255, 0)
    thickness = -1

    # Computes the coordination of the Hexbug's head in the original image.
    scaled_coordinations = compute_original_coordination(predicted, (data.iloc[sample_id].OriginalBoxCoordinationX1, data.iloc[sample_id].OriginalBoxCoordinationY1))

    # Reads and converts the original image into numpy array.
    original_sample_img = np.array(load_img(original_image_path))

    annotated_img = cv2.circle(sample_img, predicted, radius, color, thickness)

    cv2.circle(original_sample_img, scaled_coordinations, radius, color, thickness)
    
    center = [data.iloc[sample_id].CroppedHexBugCoordinationX, data.iloc[sample_id].CroppedHexBugCoordinationY]
    
    error = round(compute_error(center, predicted), 3)
    
    file_name = f'Sample ID: {sample_id}\nTruth: {center}, Predicted: {predicted}, Error: {error}'
    
    print(file_name)
    print('------------------------------------------------------------------------')
    
    if not os.path.exists('annotated_images'):
        os.mkdir('annotated_images')
    
    cv2.imshow(f'Sample ID. {sample_id}', original_sample_img)
    cv2.imwrite(f'annotated_images/{file_name}.jpg', original_sample_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 11ms/step
Sample ID: 485
Truth: [283, 116], Predicted: [282, 116], Error: 1.0
------------------------------------------------------------------------
1/1 [==============================] - 0s 13ms/step
Sample ID: 471
Truth: [157, 57], Predicted: [152, 62], Error: 7.071
------------------------------------------------------------------------
1/1 [==============================] - 0s 11ms/step
Sample ID: 55
Truth: [41, 99], Predicted: [41, 98], Error: 1.0
------------------------------------------------------------------------
1/1 [==============================] - 0s 12ms/step
Sample ID: 192
Truth: [265, 180], Predicted: [262, 178], Error: 3.606
------------------------------------------------------------------------
1/1 [==============================] - 0s 12ms/step
Sample ID: 136
Truth: [271, 30], Predicted: [270, 29], Error: 1.414
------------------------------------------------------------------------
1/1 [==============================] -